In [9]:
# !pip install transformers
# !pip install datasets
# !pip install evaluate

In [10]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, Resize, ToTensor, ToPILImage, RandomRotation, CenterCrop
from torch.utils.data import DataLoader, Dataset,random_split
from torchsummary import summary
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve



In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import evaluate
from transformers import TrainingArguments, Trainer
from torch.optim import AdamW

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [46]:
#loading/dowloading dataset
dataset = load_dataset("tweet_eval","sentiment",cache_dir="HuggingFace")

  0%|          | 0/3 [00:00<?, ?it/s]

In [47]:
#Tokenzing the text in dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased",cache_dir="HuggingFace")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [48]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [49]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))



In [50]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=4)
val_loader = DataLoader(eval_dataset, batch_size=4)

In [51]:
#loading a pretrained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3,cache_dir="HuggingFace")
# print(model)
# summary(model,input_size=(768,),batch_dim=1, dtypes=['torch.IntTensor'])
# for name, param in model.named_parameters():
#     if param.requires_grad == True:
#         print(name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [52]:
model.to(device)
num_params = sum(param.numel() for param in model.parameters())
num_trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)

In [56]:
lr=5e-5
optimizer = optim.AdamW(model.parameters(), lr=lr)

#tensorboard 
tboardWriter=SummaryWriter('runs/TextClassification-BERT')

In [57]:
!tensorboard --logdir=runs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)


In [55]:
#batch wise training loop
epochs = 2
train_losses = []
val_losses = []
best_accuracy=0
for epoch in range(epochs):  #epochs loop

    all_Y_train_epoch=np.array([]).reshape(0,1)
    all_Yhat_train_epoch=np.array([]).reshape(0,1)
    all_train_losses_epoch=np.array([])

    for batch in train_loader:        #batch wise  training on train set
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)       

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        logits = outputs.logits
             
            
        #store metrics for all batches of current epoch 
        y_hat=F.softmax(logits,dim=-1)
        y_hat=y_hat.detach().cpu().numpy()
        y_hat=np.argmax(y_hat,axis=1)
        y_hat=y_hat.reshape(-1,1)

        Y_train=batch['labels'].detach().cpu().numpy()
        Y_train=Y_train.reshape(-1,1)
        all_Y_train_epoch=np.vstack((all_Y_train_epoch,Y_train))
        all_Yhat_train_epoch=np.vstack((all_Yhat_train_epoch,y_hat))   
        all_train_losses_epoch=np.append(all_train_losses_epoch,loss.item())     
        # print("Loss= " + str(loss.item()))
    
    
    #computing metrics for current epoch
    train_losses.append(all_train_losses_epoch.mean()) #mean loss for all batches    
    acTrain=accuracy_score(all_Y_train_epoch, all_Yhat_train_epoch)
    cmTrain=confusion_matrix(all_Y_train_epoch, all_Yhat_train_epoch)
    print(cmTrain)

    #validation loop also bacth wise
    all_Y_val_epoch=np.array([]).reshape(0,1)
    all_Yhat_val_epoch=np.array([]).reshape(0,1)
    all_val_losses_epoch=np.array([])
    for batch in val_loader:  #batch wise validation set predictions only
        model.eval()
        batch = {k: v.to(device) for k, v in batch.items()}
        
        with torch.no_grad(): 
            outputs = model(**batch)       
            loss = outputs.loss
            logits = outputs.logits

        
        #store metrics for all batches of current epoch 
        y_hat_val=F.softmax(logits,dim=-1)
        y_hat_val=y_hat_val.detach().cpu().numpy()
        y_hat_val=np.argmax(y_hat_val,axis=1)
        y_hat_val=y_hat_val.reshape(-1,1)
        Y_val=batch['labels'].detach().cpu().numpy()
        Y_val=Y_val.reshape(-1,1)
        all_Y_val_epoch=np.vstack((all_Y_val_epoch,Y_val))
        all_Yhat_val_epoch=np.vstack((all_Yhat_val_epoch,y_hat_val))   
        all_val_losses_epoch=np.append(all_val_losses_epoch,loss.item())     
            

    #computing metrics for current epoch
    val_losses.append(all_val_losses_epoch.mean()) #mean loss for all batches    
    acVal=accuracy_score(all_Y_val_epoch, all_Yhat_val_epoch)
    cmVal=confusion_matrix(all_Y_val_epoch, all_Yhat_val_epoch)
    
    print(f"epoch= {epoch}, accuracyTrain= {acTrain}, accuracyVal= {acVal}, train_loss= {train_losses[epoch]}, validation_loss= {val_losses[epoch]}")

    #checkpointing training
    if(acVal>best_accuracy):
        checkpoint = {'epoch': epoch,'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),'loss': train_losses,
                      'val_loss': val_losses}
        torch.save(checkpoint,'best.pth')




    tboardWriter.add_scalar("Loss/train", train_losses[epoch], epoch)
    tboardWriter.add_scalar("Loss/val", val_losses[epoch], epoch)
    tboardWriter.add_scalar("accuracy/train", acTrain, epoch)
    tboardWriter.add_scalar("accuracy/val", acVal, epoch)


[[  5  96  55]
 [  4 315 134]
 [  2 179 210]]
epoch= 0, accuracyTrain= 0.53, accuracyVal= 0.53, train_loss= 0.965715117931366, validation_loss= 0.9294101783037185
[[ 32  85  39]
 [ 33 320 100]
 [ 11 120 260]]
epoch= 1, accuracyTrain= 0.612, accuracyVal= 0.51, train_loss= 0.8644181739091873, validation_loss= 1.0175445889234542


In [58]:
!tensorboard dev upload --logdir runs \
--name "My latest experiment" \
--description "Simple comparison of several hyperparameters"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/t4QpSAKYS1y66ZNI0OOOoQ/

[2022-11-07T05:26:19] Started scanning logdir.
[2022-11-07T05:26:20] Total uploaded: 8 scalars, 0 tensors, 0 binary objects
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 276, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.7/dist